In [0]:
!pip install psycopg2-binary

     |████████████████████████████████| 2.9MB 2.8MB/s 


In [0]:
import psycopg2

In [0]:
dbname = 'mhodwfds'
user = 'mhodwfds'
password = 'v8dvWGf-qAVfuz3f9042cAMpVX-7ZNMS'
host = 'raja.db.elephantsql.com'

In [0]:
pg_conn = psycopg2.connect(dbname=dbname, user=user,
                           password=password, host=host)

In [0]:
pg_conn

<connection object at 0x7f13c74d68d0; dsn: 'user=mhodwfds password=xxx dbname=mhodwfds host=raja.db.elephantsql.com', closed: 0>

In [0]:
pg_curs = pg_conn.cursor()

In [0]:
pg_curs.execute('SELECT * FROM test_table;')
pg_curs.fetchall()

[(1, 'A row name', None),
 (2, 'Another row, with JSON', {'a': 1, 'b': ['dog', 'cat', 42], 'c': True})]

In [0]:
!wget https://github.com/LambdaSchool/DS-Unit-3-Sprint-2-SQL-and-Databases/blob/master/module1-introduction-to-sql/rpg_db.sqlite3?raw=true

--2019-06-04 20:00:12--  https://github.com/LambdaSchool/DS-Unit-3-Sprint-2-SQL-and-Databases/blob/master/module1-introduction-to-sql/rpg_db.sqlite3?raw=true
Resolving github.com (github.com)... 192.30.253.112
Connecting to github.com (github.com)|192.30.253.112|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://github.com/LambdaSchool/DS-Unit-3-Sprint-2-SQL-and-Databases/raw/master/module1-introduction-to-sql/rpg_db.sqlite3 [following]
--2019-06-04 20:00:13--  https://github.com/LambdaSchool/DS-Unit-3-Sprint-2-SQL-and-Databases/raw/master/module1-introduction-to-sql/rpg_db.sqlite3
Reusing existing connection to github.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/LambdaSchool/DS-Unit-3-Sprint-2-SQL-and-Databases/master/module1-introduction-to-sql/rpg_db.sqlite3 [following]
--2019-06-04 20:00:13--  https://raw.githubusercontent.com/LambdaSchool/DS-Unit-3-Sprint-2-SQL-and-Databases/master/module

In [0]:
!mv 'rpg_db.sqlite3?raw=true' rpg_db.sqlite3

In [0]:
import sqlite3
sl_conn = sqlite3.connect('rpg_db.sqlite3')
sl_curs = sl_conn.cursor()

In [0]:
sl_curs.execute('SELECT COUNT(*) FROM charactercreator_character;').fetchall()

[(302,)]

In [0]:
characters = sl_curs.execute('SELECT * FROM charactercreator_character;').fetchall()

In [0]:
len(characters)

302

In [0]:
characters[0]

(1, 'Aliquid iste optio reiciendi', 0, 0, 10, 1, 1, 1, 1)

In [0]:
characters[-1]

(302, 'Aliquam n', 0, 0, 10, 1, 1, 1, 1)

```python
class Character(models.Model):
    """Base representation of RPG characters."""
    character_id = models.AutoField(primary_key=True)
    name = models.CharField("Character's name", max_length=30)
    level = models.IntegerField(default=0)
    exp = models.IntegerField(default=0)
    hp = models.IntegerField(default=10)
    strength = models.IntegerField(default=1)
    intelligence = models.IntegerField(default=1)
    dexterity = models.IntegerField(default=1)
    wisdom = models.IntegerField(default=1)
    inventory = models.ManyToManyField(Item, verbose_name="Items the character has")
```

In [0]:
# We have the characters in local Python object
# (Specifically, a list of tuples)
# Next - CREATE TABLE in PostgreSQL
# In other words - make the Character schema!

create_character_table = """
CREATE TABLE charactercreator_character (
  character_id SERIAL PRIMARY KEY,
  name VARCHAR(30),
  level INT,
  exp INT,
  hp INT,
  strength INT,
  intelligence INT,
  dexterity INT,
  wisdom INT
)
"""

In [0]:
pg_curs.execute(create_character_table)

In [0]:
characters[0]

(1, 'Aliquid iste optio reiciendi', 0, 0, 10, 1, 1, 1, 1)

In [0]:
str(characters[0])

"(1, 'Aliquid iste optio reiciendi', 0, 0, 10, 1, 1, 1, 1)"

In [0]:
attempted_insert = """
  INSERT INTO charactercreator_character
  VALUES """ + str(characters[0])
attempted_insert

"\n  INSERT INTO charactercreator_character\n  VALUES (1, 'Aliquid iste optio reiciendi', 0, 0, 10, 1, 1, 1, 1)"

In [0]:
pg_curs.execute(attempted_insert)

In [0]:
pg_conn.commit()  # We need to "save" our work for the new table/data to show up

In [0]:
pg_curs = pg_conn.cursor()
pg_curs.execute('SELECT * FROM charactercreator_character;')
pg_characters = pg_curs.fetchall()

In [0]:
pg_characters[0]

(1, 'Aliquid iste optio reiciendi', 0, 0, 10, 1, 1, 1, 1)

In [0]:
len(pg_characters)

1

In [0]:
# Let's insert the rest of the characters!
for character in characters[1:]:
  insert_character = """
  INSERT INTO charactercreator_character
  VALUES """ + str(character)
  pg_curs.execute(insert_character)
pg_conn.commit()

In [0]:
pg_curs = pg_conn.cursor()
pg_curs.execute('SELECT * FROM charactercreator_character;')
pg_characters = pg_curs.fetchall()

In [0]:
len(pg_characters)

302

In [0]:
pg_characters[0]

(1, 'Aliquid iste optio reiciendi', 0, 0, 10, 1, 1, 1, 1)

In [0]:
pg_characters[-1]

(302, 'Aliquam n', 0, 0, 10, 1, 1, 1, 1)

In [0]:
len(characters)

302

In [0]:
for character, pg_character in zip(characters, pg_characters):
  assert character == pg_character

In [0]:
# Nothing happened! That means the assert was always happy, and they're all the same!